In [11]:
import socket, threading, pickle, uuid, os
import numpy as np
BUFSIZE = 1024
DEBUG_NETWORK = True
class UnixUdpServer:
    def __init__(self, unix_path, obj='bytes') -> None:
        try: os.makedirs(os.path.dirname(unix_path))
        except: pass
        self.unix_path = unix_path
        self.server = socket.socket(socket.AF_UNIX, socket.SOCK_DGRAM)
        self.server.bind(self.unix_path)
        self.most_recent_client = None
        self.use_pickle = (obj=='pickle')
        self.convert_str = (obj=='str')
        return

    def wait_next_dgram(self):
        data, self.most_recent_client = self.server.recvfrom(BUFSIZE)
        print('self.most_recent_client',self.most_recent_client)
        if self.convert_str: data = data.decode('utf8')
        if self.use_pickle: data = pickle.loads(data)
        if DEBUG_NETWORK: print('recv from :', self.most_recent_client, ' data :', data)
        return data

    def reply_last_client(self, data):
        assert self.most_recent_client is not None
        if DEBUG_NETWORK: print('reply_last_client :', self.most_recent_client, ' data :', data)
        if self.use_pickle: data = pickle.dumps(data)
        if self.convert_str: data = bytes(data, encoding='utf8')
        self.server.sendto(data, self.most_recent_client)
        return

    def __del__(self):
        self.server.close()
        return

class UnixUdpTargetedClient:
    def __init__(self, target_unix_path, self_unix_path=None, obj='bytes') -> None:
        self.target_unix_path = target_unix_path
        if self_unix_path is not None:
            self.self_unix_path = self_unix_path  
        else:
            self.self_unix_path = target_unix_path+'_client_'+uuid.uuid1().hex[:5]
        self.client = socket.socket(socket.AF_UNIX, socket.SOCK_DGRAM)
        self.client.bind(self.self_unix_path)
        self.use_pickle = (obj=='pickle')
        self.convert_str = (obj=='str')
        return

    def send_dgram_to_target(self, data):
        if self.use_pickle: data = pickle.dumps(data)
        if self.convert_str: data = bytes(data, encoding='utf8')
        self.client.sendto(data, self.target_unix_path)
        if DEBUG_NETWORK: print('send_targeted_dgram :', self.target_unix_path, ' data :', data)
        return

    def send_and_wait_reply(self, data):
        if self.use_pickle: data = pickle.dumps(data)
        if self.convert_str: data = bytes(data, encoding='utf8')
        self.client.sendto(data, self.target_unix_path)
        data, _ = self.client.recvfrom(BUFSIZE)
        if self.convert_str: data = data.decode('utf8')
        if self.use_pickle: data = pickle.loads(data)
        if DEBUG_NETWORK: print('get_reply :', self.target_unix_path, ' data :', data)
        return data

remote_uuid = uuid.uuid1().hex   # use uuid to identify threads

unix_path = 'RECYCLE/Sockets/unix/%s'%remote_uuid
server = UnixUdpServer(unix_path, obj='pickle')
client = UnixUdpTargetedClient(unix_path, obj='pickle')

def server_fn():
    data = server.wait_next_dgram()
    server.reply_last_client(np.array([4,5,6]))

def client_fn():
    rep = client.send_and_wait_reply(np.array([1,2,3]))


thread_hi = threading.Thread(target=server_fn)
thread_hello = threading.Thread(target=client_fn)
# 启动线程
thread_hi.start()
thread_hello.start()





self.most_recent_client RECYCLE/Sockets/unix/f0f17a80ad0911ec9e623cecef04a859_client_f0f1e
recv from : RECYCLE/Sockets/unix/f0f17a80ad0911ec9e623cecef04a859_client_f0f1e  data : [1 2 3]
reply_last_client : RECYCLE/Sockets/unix/f0f17a80ad0911ec9e623cecef04a859_client_f0f1e  data : [4 5 6]
get_reply : RECYCLE/Sockets/unix/f0f17a80ad0911ec9e623cecef04a859  data : [4 5 6]


In [8]:
unix_path

'RECYCLE/Sockets/9a64f35ead0911ec9e623cecef04a859'

client.send_targeted_dgram('ddd')

In [ ]:
import torch
# from models import BaseVAE
from torch import nn
from torch.nn import functional as F
from torch import nn, Tensor
# from abc import abstractmethod


class VanillaVAE(nn.Module):

    def __init__(self,
                 input_dim: int,
                 latent_dim: int,
                 hidden_dim: int,
                 degenerate2ae = False,
                 **kwargs) -> None:
        super(VanillaVAE, self).__init__()
        self.training = True

        # Part 1, encoder
        self.FC_input = nn.Linear(input_dim, hidden_dim)
        self.FC_input2 = nn.Linear(hidden_dim, hidden_dim)
        self.FC_mean  = nn.Linear(hidden_dim, latent_dim)
        self.FC_var   = nn.Linear (hidden_dim, latent_dim)
        
        self.LeakyReLU = nn.LeakyReLU(0.2)
        
        # Part 2, decoder
        self.FC_hidden = nn.Linear(latent_dim, hidden_dim)
        self.FC_hidden2 = nn.Linear(hidden_dim, hidden_dim)
        self.FC_output = nn.Linear(hidden_dim, input_dim)
        
        self.LeakyReLU = nn.LeakyReLU(0.2)
        
        self.degenerate2ae = degenerate2ae

    def encode(self, input: Tensor):
        """
        Encodes the input by passing through the encoder network
        and returns the latent codes.
        :param input: (Tensor) Input tensor to encoder
        :return: (Tensor) List of latent codes
        """
        h_       = self.LeakyReLU(self.FC_input(input))
        h_       = self.LeakyReLU(self.FC_input2(h_))
        mean     = self.FC_mean(h_)
        log_var  = self.FC_var(h_) 

        return mean, log_var

    def decode(self, z: Tensor) -> Tensor:
        """
        Maps the given latent codes
        onto the image space.
        :param z: (Tensor) [B x D]
        :return: (Tensor) [B x C x H x W]
        """
        h     = self.LeakyReLU(self.FC_hidden(z))
        h     = self.LeakyReLU(self.FC_hidden2(h))
        x_hat = self.FC_output(h)

        return x_hat

    def reparameterize(self, mu: Tensor, logvar: Tensor) -> Tensor:
        """
        Reparameterization trick to sample from N(mu, var) from
        N(0,1).
        :param mu: (Tensor) Mean of the latent Gaussian [B x D]
        :param logvar: (Tensor) Standard deviation of the latent Gaussian [B x D]
        :return: (Tensor) [B x D]
        """
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return eps * std + mu

    def forward(self, input: Tensor, **kwargs):
        mu, log_var = self.encode(input)
        if not self.degenerate2ae:
            z = self.reparameterize(mu, log_var)
        else:
            z = mu
        return  [self.decode(z), input, mu, log_var]    #  x_hat, mean, log_var

    def loss_function(self,
                      x, x_hat, mean, log_var, kld_loss_weight, recons_loss_weight,
                      **kwargs) -> dict:
        """
        Computes the VAE loss function.
        KL(N(\mu, \sigma), N(0, 1)) = \log \frac{1}{\sigma} + \frac{\sigma^2 + \mu^2}{2} - \frac{1}{2}
        :param args:
        :param kwargs:
        :return:
        """
        recons = x_hat
        input = x
        mu = mean
        log_var = log_var

        # kld_weight = kwargs['M_N'] # Account for the minibatch samples from the dataset
        recons_loss =F.mse_loss(recons, input)

        # 计算高斯分布和标准正态分布的KL散度
        if not self.degenerate2ae:
            kld_loss = torch.mean(-0.5 * torch.sum(1 + log_var - mu ** 2 - log_var.exp(), dim = 1), dim = 0)
            loss = recons_loss*recons_loss_weight + kld_loss*kld_loss_weight
        else:
            kld_loss = torch.zeros_like(recons_loss)
            loss = recons_loss

        return {'loss': loss, 'Reconstruction_Loss':recons_loss.detach(), 'KLD':-kld_loss.detach()}


    # Do I need this?
    def sample(self,
               num_samples:int,
               current_device: int, **kwargs) -> Tensor:
        """
        Samples from the latent space and return the corresponding
        image space map.
        :param num_samples: (Int) Number of samples
        :param current_device: (Int) Device to run the model
        :return: (Tensor)
        """
        z = torch.randn(num_samples,
                        self.latent_dim)

        z = z.to(current_device)

        samples = self.decode(z)
        return samples

    # Do I need this?
    def generate(self, x: Tensor, **kwargs) -> Tensor:
        """
        Given an input image x, returns the reconstructed image
        :param x: (Tensor) [B x C x H x W]
        :return: (Tensor) [B x C x H x W]
        """

        return self.forward(x)[0]

In [ ]:
import numpy as np
from UTILS.tensor_ops import _2tensor, cfg
cfg.device_ = 'cuda:0'
cfg.use_float64_ = False
cfg.init = True

input_dim = 10
my_data_sample_x = np.random.rand(1000,10)*10 - 1 # random from -1 to +1
my_data_sample_y = my_data_sample_x.mean(-1)**2 + my_data_sample_x.mean(-1)*(-2) + 1

# solve the weight inital problem
vae_mod = VanillaVAE(input_dim=input_dim, latent_dim=16, hidden_dim=32, degenerate2ae=False)
my_data_sample_x = _2tensor(my_data_sample_x)
vae_mod = _2tensor(vae_mod)
from torch.optim import Adam
optimizer = Adam(vae_mod.parameters(), lr=3e-3)

print("Start training VAE...")
vae_mod.train()

for epoch in range(5000):
    overall_loss = 0
    x = my_data_sample_x

    optimizer.zero_grad()

    #  [self.decode(z), input, mu, log_var]    #  x_hat, mean, log_var
    x_hat, x_origin, mean, log_var = vae_mod(x) # model(x)
    error = ( torch.abs(x_hat)-torch.abs(x) )/( torch.abs(x)+1e-9 )
    std_, mean_ = torch.std_mean(error, unbiased=False)
    print('\test error mean %.2f and std %.2f'%(mean_.item(), std_.item()) )


    lossdict = vae_mod.loss_function(x=x, x_hat=x_hat, mean=mean, log_var=log_var, kld_loss_weight=1, recons_loss_weight=1)
    loss = lossdict['loss']
    overall_loss += loss.item()

    loss.backward()
    optimizer.step()
    # 'Reconstruction_Loss':recons_loss.detach(), 'KLD'
    print("\tEpoch", epoch + 1, "complete!", "\tReconstruction_Loss: ", lossdict['Reconstruction_Loss'].item(), "\tKLD: ", lossdict['KLD'].item())

print("Finish!!")

In [ ]:
import torch
# from models import BaseVAE
from torch import nn
from torch.nn import functional as F
from torch import nn, Tensor
# from abc import abstractmethod


class VanillaVAE(nn.Module):

    def __init__(self,
                 input_dim: int,
                 latent_dim: int,
                 hidden_dim: int,
                 degenerate2ae = False,
                 **kwargs) -> None:
        super(VanillaVAE, self).__init__()
        self.training = True

        # Part 1, encoder
        self.FC_input = nn.Linear(input_dim, hidden_dim)
        self.FC_input2 = nn.Linear(hidden_dim, hidden_dim)
        self.FC_mean  = nn.Linear(hidden_dim, latent_dim)
        self.FC_var   = nn.Linear (hidden_dim, latent_dim)
        
        self.LeakyReLU = nn.LeakyReLU(0.2)
        
        # Part 2, decoder
        self.FC_hidden = nn.Linear(latent_dim, hidden_dim)
        self.FC_hidden2 = nn.Linear(hidden_dim, hidden_dim)
        self.FC_output = nn.Linear(hidden_dim, input_dim)
        
        self.LeakyReLU = nn.LeakyReLU(0.2)
        
        self.degenerate2ae = degenerate2ae

    def encode(self, input: Tensor):
        """
        Encodes the input by passing through the encoder network
        and returns the latent codes.
        :param input: (Tensor) Input tensor to encoder
        :return: (Tensor) List of latent codes
        """
        h_       = self.LeakyReLU(self.FC_input(input))
        h_       = self.LeakyReLU(self.FC_input2(h_))
        mean     = self.FC_mean(h_)
        log_var  = self.FC_var(h_) 

        return mean, log_var

    def decode(self, z: Tensor) -> Tensor:
        """
        Maps the given latent codes
        onto the image space.
        :param z: (Tensor) [B x D]
        :return: (Tensor) [B x C x H x W]
        """
        h     = self.LeakyReLU(self.FC_hidden(z))
        h     = self.LeakyReLU(self.FC_hidden2(h))
        x_hat = self.FC_output(h)

        return x_hat

    def reparameterize(self, mu: Tensor, logvar: Tensor) -> Tensor:
        """
        Reparameterization trick to sample from N(mu, var) from
        N(0,1).
        :param mu: (Tensor) Mean of the latent Gaussian [B x D]
        :param logvar: (Tensor) Standard deviation of the latent Gaussian [B x D]
        :return: (Tensor) [B x D]
        """
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return eps * std + mu

    def forward(self, input: Tensor, **kwargs):
        mu, log_var = self.encode(input)
        if not self.degenerate2ae:
            z = self.reparameterize(mu, log_var)
        else:
            z = mu
        return  [self.decode(z), input, mu, log_var]    #  x_hat, mean, log_var

    def loss_function(self,
                      x, x_hat, mean, log_var, kld_loss_weight, recons_loss_weight,
                      **kwargs) -> dict:
        """
        Computes the VAE loss function.
        KL(N(\mu, \sigma), N(0, 1)) = \log \frac{1}{\sigma} + \frac{\sigma^2 + \mu^2}{2} - \frac{1}{2}
        :param args:
        :param kwargs:
        :return:
        """
        recons = x_hat
        input = x
        mu = mean
        log_var = log_var

        # kld_weight = kwargs['M_N'] # Account for the minibatch samples from the dataset
        recons_loss = F.mse_loss(recons, input)

        # 计算高斯分布和标准正态分布的KL散度
        if not self.degenerate2ae:
            kld_loss = torch.mean(-0.5 * torch.sum(1 + log_var - mu ** 2 - log_var.exp(), dim = 1), dim = 0)
            loss = recons_loss*recons_loss_weight + kld_loss*kld_loss_weight
        else:
            kld_loss = torch.zeros_like(recons_loss)
            loss = recons_loss

        return {'loss': loss, 'Reconstruction_Loss':recons_loss.detach(), 'KLD':-kld_loss.detach()}


    # Do I need this?
    def sample(self,
               num_samples:int,
               current_device: int, **kwargs) -> Tensor:
        """
        Samples from the latent space and return the corresponding
        image space map.
        :param num_samples: (Int) Number of samples
        :param current_device: (Int) Device to run the model
        :return: (Tensor)
        """
        z = torch.randn(num_samples,
                        self.latent_dim)

        z = z.to(current_device)

        samples = self.decode(z)
        return samples

    # Do I need this?
    def generate(self, x: Tensor, **kwargs) -> Tensor:
        """
        Given an input image x, returns the reconstructed image
        :param x: (Tensor) [B x C x H x W]
        :return: (Tensor) [B x C x H x W]
        """

        return self.forward(x)[0]

import numpy as np
from UTILS.tensor_ops import _2tensor, cfg
cfg.device_ = 'cuda:0'
cfg.use_float64_ = False
cfg.init = True

input_dim = 10
my_data_sample_x = (np.random.rand(1000,10)-0.5)*2 + 5 # 0.75~1.25
my_data_sample_y = my_data_sample_x.mean(-1)**2 + my_data_sample_x.mean(-1)*(-2) + 1

# solve the weight inital problem
vae_mod = VanillaVAE(input_dim=input_dim, latent_dim=16, hidden_dim=32, degenerate2ae=False)
my_data_sample_x = _2tensor(my_data_sample_x)
vae_mod = _2tensor(vae_mod)
from torch.optim import Adam
optimizer = Adam(vae_mod.parameters(), lr=3e-3)

print("Start training VAE...")
vae_mod.train()

for epoch in range(5000):
    overall_loss = 0
    x = my_data_sample_x

    optimizer.zero_grad()

    #  [self.decode(z), input, mu, log_var]    #  x_hat, mean, log_var
    x_hat, x_origin, mean, log_var = vae_mod(x) # model(x)
    error = ( torch.abs(x_hat - x) )/( torch.abs(x)+1e-9 )
    std_, mean_ = torch.std_mean(error, unbiased=False)
    print('\test error mean %.2f%% and std %.2f'%(mean_.item()*100, std_.item()) , end='\t')


    lossdict = vae_mod.loss_function(x=x, x_hat=x_hat, mean=mean, log_var=log_var, kld_loss_weight=1, recons_loss_weight=1)
    loss = lossdict['loss']
    overall_loss += loss.item()

    loss.backward()
    optimizer.step()
    # 'Reconstruction_Loss':recons_loss.detach(), 'KLD'
    print("\tEpoch", epoch + 1, "complete!", "\tReconstruction_Loss: ", lossdict['Reconstruction_Loss'].item(), "\tKLD: ", lossdict['KLD'].item())

print("Finish!!")